In [1]:
import numpy as np
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
import scanpy as sc
import scipy

In [2]:
adata = ad.read_h5ad("/Users/apple/Desktop/KB/data/LarryData/Larry_41093_2000_norm_log.h5ad")

In [3]:
adata.shape

(41093, 2000)

In [4]:
adata.obs

,Library,Cell barcode,time_info,Starting population,state_info,Well,SPRING-x,SPRING-y,Time point,clone_id,n_genes
d6_2_2:GCGTGCAAAGAAGTTA,d6_2_2,GCGTGCAA-AGAAGTTA,6.0,Lin-Kit+Sca1-,Undifferentiated,2,411.496,-96.190,6,573,4050
d6_2_2:AAGGGACCCTCGATGC,d6_2_2,AAGGGACC-CTCGATGC,6.0,Lin-Kit+Sca1-,Undifferentiated,2,-587.462,-306.925,6,1440,3139
d6_2_2:CGTACCGAAGCGCCTT,d6_2_2,CGTACCGA-AGCGCCTT,6.0,Lin-Kit+Sca1-,Monocyte,2,1429.805,-429.300,6,394,2403
d6_2_2:CCGTAGCTAGGCAGTT,d6_2_2,CCGTAGCT-AGGCAGTT,6.0,Lin-Kit+Sca1-,Undifferentiated,2,-1169.594,362.010,6,1972,1608
d6_2_2:TAGGCTTTCGTGGATA,d6_2_2,TAGGCTTT-CGTGGATA,6.0,Lin-Kit+Sca1-,Neutrophil,2,1236.039,-2113.013,6,1450,1667
...,...,...,...,...,...,...,...,...,...,...,...
LSK_d6_1_3:CTGTGACCTGCTATTT,LSK_d6_1_3,CTGTGACC-TGCTATTT,6.0,Lin-Kit+Sca1+,Undifferentiated,1,307.879,-27.451,6,4999,2432
LSK_d6_1_3:AGGCGTTTCCTATTCA,LSK_d6_1_3,AGGCGTTT-CCTATTCA,6.0,Lin-Kit+Sca1+,Undifferentiated,1,-1074.917,441.981,6,12,1876
LSK_d6_1_3:CGTGCAACTGGCCTGT,LSK_d6_1_3,CGTGCAAC-TGGCCTGT,6.0,Lin-Kit+Sca1+,Undifferentiated,1,247.634,281.262,6,267,1434
LSK_d6_1_3:CCCTTGCAAGTGAAAG,LSK_d6_1_3,CCCTTGCA-AGTGAAAG,6.0,Lin-Kit+Sca1+,Neutrophil,1,353.734,-980.246,6,3534,1252


In [5]:
# Step 1: Identify unique clone_ids and their frequencies
clone_id_counts = adata.obs['clone_id'].value_counts()

In [6]:


# Step 2: Initialize empty lists to hold indices for adata_train and adata_test
test_indices = []
train_indices = []

# Step 3: Loop through each clone_id and split based on the criteria
for clone_id, count in clone_id_counts.items():
    clone_indices = adata.obs[adata.obs['clone_id'] == clone_id].index
    
    if count >= 10:
        # Select 10% of the elements randomly for adata_test
        test_size = int(np.ceil(0.1 * count))
        test_clone_indices = np.random.choice(clone_indices, size=test_size, replace=False)
        test_indices.extend(test_clone_indices)
        
        # The remaining elements go to adata_train
        train_clone_indices = list(set(clone_indices) - set(test_clone_indices))
        train_indices.extend(train_clone_indices)
    else:
        # All elements go to adata_train
        train_indices.extend(clone_indices)



In [7]:

# Step 4: Subset the original adata to create adata_train and adata_test
adata_train = adata[train_indices, :].copy()
adata_test = adata[test_indices, :].copy()


In [8]:
adata_test.shape, adata_train.shape

((3886, 2000), (37207, 2000))

In [10]:
adata_test.write_h5ad('Larry_test.h5ad')
adata_train.write_h5ad('Larry_train.h5ad')

